## 🌐 Connect Colab to Google Drive

In [ ]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/Homework 1_lst
%ls

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/Homework 1_lst
densenet_base.keras              model.png
densenet_model_ft3.keras         model.py
densenet_model_ft4.keras         submission_241123_160630.zip
Homework_1_dataset_augmentation  training_augmented_big.npz
Homework_1_densenet_lion         training_augmented_big_randaugment.npz
Homework_1_densenet_lion_def     training_set.npz
Homework_1_randAugment           validation_test.npz
inception_model_ft1.keras        weights.keras
inception_model_ft2.keras


## ⚙️ Import Libraries

In [ ]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

!pip install keras-cv
import keras_cv

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import requests
from io import BytesIO
import cv2
from PIL import Image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)

%matplotlib inline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 32.9 MB/s eta 0:00:00
2.17.1


## ⏳ Load the Data

In [ ]:
data = np.load('training_augmented_big.npz')
X_train = data['X_train']
y_train = data['y_train']
print(X_train.shape, y_train.shape)

data = np.load('validation_test.npz')
X_val = data['X_val']
y_val = data['y_val']
print(X_val.shape, y_val.shape)
X_test = data['X_test']
y_test = data['y_test']
print(X_test.shape, y_test.shape)

(75243, 96, 96, 3) (75243, 8)
(3588, 96, 96, 3) (3588, 8)
(1794, 96, 96, 3) (1794, 8)


## ⚓ DenseNet for Transfer Learning

In [ ]:
from tensorflow.keras.applications import DenseNet121

# Initialise DenseNet model with pretrained weights, for transfer learning
densenet = DenseNet121(
    input_shape=(96, 96, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze all layers to use it solely as a feature extractor
densenet.trainable = False

# Define input layer with shape matching the input images
inputs = tfk.Input(shape=(96, 96, 3), name='input_layer')

# Apply data augmentation for training robustness
augmentation = tf.keras.Sequential([
    tfkl.RandomFlip("horizontal_vertical"),
    tfkl.RandomTranslation(0.25, 0.25),
    tfkl.RandomZoom(0.4),
    tfkl.RandomRotation(0.6),
    tfkl.RandomContrast(0.3),
    tfkl.RandomBrightness(0.4)
], name='preprocessing')

x = augmentation(inputs)

# Rescaling
x = tfkl.Rescaling(1./255)(x)

# Pass augmented inputs through the denseNet feature extractor
x = densenet(x)

# Add a dropout layer for regularisation
x = tfkl.Dropout(0.4, name='dropout')(x)

# Add final Dense layer for classification with softmax activation
outputs = tfkl.Dense(y_train.shape[-1], activation='softmax', name='dense')(x)

# Define the complete model linking input and output
inception_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Lion configuratin:
lion_optimizer = tfk.optimizers.Lion(
    learning_rate = 0.001,
    weight_decay = 0.01,
)

# Compile with Lion
inception_model.compile(
    optimizer=lion_optimizer,
    loss=tfk.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Display a summary of the model architecture
densenet.summary(expand_nested=False)

# Display model architecture with layer shapes and trainable parameters
tfk.utils.plot_model(inception_model, expand_nested=True, show_trainable=True,
                     show_shapes=True, dpi=70)

## 😓 Training

In [ ]:
# Against overfitting
early_stopping_inception = tfk.callbacks.EarlyStopping(
    monitor = 'val_accuracy',
    mode = 'max',
    patience = 10,
    restore_best_weights=True
)

# Store the callback in a list
callbacks = [early_stopping_inception]

In [ ]:
# Train the model
inception_history = inception_model.fit(
    x = X_train,
    y = y_train,
    batch_size = 128,
    epochs = 300,
    validation_data = (X_val, y_val),
    callbacks = callbacks,
    shuffle = True
).history

# Calculate and print the best validation accuracy achieved
final_val_accuracy = round(max(inception_history['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
model_filename = 'densenet_base.keras'
inception_model.save(model_filename)

# Free memory by deleting the model instance
del inception_model

Epoch 1/300
588/588 ━━━━━━━━━━━━━━━━━━━━ 37s 36ms/step - accuracy: 0.4430 - loss: 1.8045 - val_accuracy: 0.8113 - val_loss: 0.6191
Epoch 2/300
588/588 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.5269 - loss: 1.4586 - val_accuracy: 0.8333 - val_loss: 0.5705
Epoch 3/300
588/588 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.5320 - loss: 1.4342 - val_accuracy: 0.8350 - val_loss: 0.5443
Epoch 4/300
588/588 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.5349 - loss: 1.4423 - val_accuracy: 0.8055 - val_loss: 0.6112
Epoch 5/300
588/588 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.5408 - loss: 1.4227 - val_accuracy: 0.8225 - val_loss: 0.5716
Epoch 6/300
588/588 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.5412 - loss: 1.4177 - val_accuracy: 0.8364 - val_loss: 0.5651
Epoch 7/300
588/588 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.5435 - loss: 1.4060 - val_accuracy: 0.8250 - val_loss: 0.5688
Epoch 8/300
588/588 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - accuracy: 0.5404 - loss: 1

## ⏰ First fine tuning

In [ ]:
# Re-load the model
inception_model = tfk.models.load_model('densenet_base.keras')

print(len(inception_model.get_layer('densenet121').layers))

for layer in inception_model.layers:
    print(layer.name, type(layer).__name__)

# Set the model layers as trainable
inception_model.get_layer('densenet121').trainable = True

# Set all layers as non-trainable
for layer in inception_model.get_layer('densenet121').layers:
    layer.trainable = False

# Enable training only for Conv2D and DepthwiseConv2D layers
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

428
input_layer InputLayer
preprocessing Sequential
rescaling Rescaling
densenet121 Functional
dropout Dropout
dense Dense
2 conv1_conv Conv2D True
9 conv2_block1_1_conv Conv2D True
12 conv2_block1_2_conv Conv2D True
16 conv2_block2_1_conv Conv2D True
19 conv2_block2_2_conv Conv2D True
23 conv2_block3_1_conv Conv2D True
26 conv2_block3_2_conv Conv2D True
30 conv2_block4_1_conv Conv2D True
33 conv2_block4_2_conv Conv2D True
37 conv2_block5_1_conv Conv2D True
40 conv2_block5_2_conv Conv2D True
44 conv2_block6_1_conv Conv2D True
47 conv2_block6_2_conv Conv2D True
51 pool2_conv Conv2D True
55 conv3_block1_1_conv Conv2D True
58 conv3_block1_2_conv Conv2D True
62 conv3_block2_1_conv Conv2D True
65 conv3_block2_2_conv Conv2D True
69 conv3_block3_1_conv Conv2D True
72 conv3_block3_2_conv Conv2D True
76 conv3_block4_1_conv Conv2D True
79 conv3_block4_2_conv Conv2D True
83 conv3_block5_1_conv Conv2D True
86 conv3_block5_2_conv Conv2D True
90 conv3_block6_1_conv Conv2D True
93 conv3_block6_2_conv

In [ ]:
# Set the number of layers to freeze (beginning)
N = 350

# Set the first N layers as non-trainable
for i, layer in enumerate(inception_model.get_layer('densenet121').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    print(i, layer.name, layer.trainable)

0 input_layer False
1 zero_padding2d False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 zero_padding2d_1 False
6 pool1 False
7 conv2_block1_0_bn False
8 conv2_block1_0_relu False
9 conv2_block1_1_conv False
10 conv2_block1_1_bn False
11 conv2_block1_1_relu False
12 conv2_block1_2_conv False
13 conv2_block1_concat False
14 conv2_block2_0_bn False
15 conv2_block2_0_relu False
16 conv2_block2_1_conv False
17 conv2_block2_1_bn False
18 conv2_block2_1_relu False
19 conv2_block2_2_conv False
20 conv2_block2_concat False
21 conv2_block3_0_bn False
22 conv2_block3_0_relu False
23 conv2_block3_1_conv False
24 conv2_block3_1_bn False
25 conv2_block3_1_relu False
26 conv2_block3_2_conv False
27 conv2_block3_concat False
28 conv2_block4_0_bn False
29 conv2_block4_0_relu False
30 conv2_block4_1_conv False
31 conv2_block4_1_bn False
32 conv2_block4_1_relu False
33 conv2_block4_2_conv False
34 conv2_block4_concat False
35 conv2_block5_0_bn False
36 conv2_block5_0_relu False
37 conv2_block

In [ ]:
# Compile
inception_model.compile(
    optimizer=tfk.optimizers.Nadam(5e-4),
    loss=tfk.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
# The "for" iteration is paramount to face the colab limited resources and gave us the oportunity
# to split the training in different phases saving the partial results iteratively
for i in range(3):
  # Fine-tune the model
  inception_history = inception_model.fit(
      x = X_train,
      y = y_train,
      batch_size = 64,
      epochs = 30,
      validation_data = (X_val, y_val),
      callbacks=[
          tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True),
          tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
      ],
      shuffle = True
  ).history

  # Calculate and print the final validation accuracy
  final_val_accuracy = round(max(inception_history['val_accuracy'])* 100, 2)
  print(f'Final validation accuracy: {final_val_accuracy}%')

  # Save the trained model to a file with the accuracy included in the filename
  model_filename = 'inception_model_ft1.keras'
  inception_model.save(model_filename)

# Delete the model to free up resources
del inception_model

Epoch 1/30
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 62s 37ms/step - accuracy: 0.6432 - loss: 1.0723 - val_accuracy: 0.9122 - val_loss: 0.2577 - learning_rate: 5.0000e-04
Epoch 2/30
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 37s 32ms/step - accuracy: 0.7483 - loss: 0.7704 - val_accuracy: 0.9214 - val_loss: 0.2245 - learning_rate: 5.0000e-04
Epoch 3/30
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 37s 32ms/step - accuracy: 0.7718 - loss: 0.7045 - val_accuracy: 0.9381 - val_loss: 0.1893 - learning_rate: 5.0000e-04
Epoch 4/30
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 37s 32ms/step - accuracy: 0.7843 - loss: 0.6722 - val_accuracy: 0.9415 - val_loss: 0.1835 - learning_rate: 5.0000e-04
Epoch 5/30
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 37s 32ms/step - accuracy: 0.7916 - loss: 0.6473 - val_accuracy: 0.9326 - val_loss: 0.1994 - learning_rate: 5.0000e-04
Epoch 6/30
1176/1176 ━━━━━━━━━━━━━━━━━━━━ 37s 32ms/step - accuracy: 0.7985 - loss: 0.6294 - val_accuracy: 0.9518 - val_loss: 0.1461 - learning_rate: 5.0000e-04
Epoch 7/30
1176/1176 ━━━━━━━━━━━━━━━━━━━

## ⛳ Second fine tuning

In [ ]:
# Re-load the model
inception_model = tfk.models.load_model('inception_model_ft1.keras')

for layer in inception_model.layers:
    print(layer.name, type(layer).__name__)

# Set the model layers as trainable
inception_model.get_layer('densenet121').trainable = True

# Set all layers as non-trainable
for layer in inception_model.get_layer('densenet121').layers:
    layer.trainable = False

# Enable training only for Conv2D and DepthwiseConv2D layers
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

input_layer InputLayer
preprocessing Sequential
rescaling Rescaling
densenet121 Functional
dropout Dropout
dense Dense
2 conv1_conv Conv2D True
9 conv2_block1_1_conv Conv2D True
12 conv2_block1_2_conv Conv2D True
16 conv2_block2_1_conv Conv2D True
19 conv2_block2_2_conv Conv2D True
23 conv2_block3_1_conv Conv2D True
26 conv2_block3_2_conv Conv2D True
30 conv2_block4_1_conv Conv2D True
33 conv2_block4_2_conv Conv2D True
37 conv2_block5_1_conv Conv2D True
40 conv2_block5_2_conv Conv2D True
44 conv2_block6_1_conv Conv2D True
47 conv2_block6_2_conv Conv2D True
51 pool2_conv Conv2D True
55 conv3_block1_1_conv Conv2D True
58 conv3_block1_2_conv Conv2D True
62 conv3_block2_1_conv Conv2D True
65 conv3_block2_2_conv Conv2D True
69 conv3_block3_1_conv Conv2D True
72 conv3_block3_2_conv Conv2D True
76 conv3_block4_1_conv Conv2D True
79 conv3_block4_2_conv Conv2D True
83 conv3_block5_1_conv Conv2D True
86 conv3_block5_2_conv Conv2D True
90 conv3_block6_1_conv Conv2D True
93 conv3_block6_2_conv Con

In [ ]:
# Set the number of layers to freeze
N = 200

# Set the first N layers as non-trainable
for i, layer in enumerate(inception_model.get_layer('densenet121').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    print(i, layer.name, layer.trainable)

0 input_layer False
1 zero_padding2d False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 zero_padding2d_1 False
6 pool1 False
7 conv2_block1_0_bn False
8 conv2_block1_0_relu False
9 conv2_block1_1_conv False
10 conv2_block1_1_bn False
11 conv2_block1_1_relu False
12 conv2_block1_2_conv False
13 conv2_block1_concat False
14 conv2_block2_0_bn False
15 conv2_block2_0_relu False
16 conv2_block2_1_conv False
17 conv2_block2_1_bn False
18 conv2_block2_1_relu False
19 conv2_block2_2_conv False
20 conv2_block2_concat False
21 conv2_block3_0_bn False
22 conv2_block3_0_relu False
23 conv2_block3_1_conv False
24 conv2_block3_1_bn False
25 conv2_block3_1_relu False
26 conv2_block3_2_conv False
27 conv2_block3_concat False
28 conv2_block4_0_bn False
29 conv2_block4_0_relu False
30 conv2_block4_1_conv False
31 conv2_block4_1_bn False
32 conv2_block4_1_relu False
33 conv2_block4_2_conv False
34 conv2_block4_concat False
35 conv2_block5_0_bn False
36 conv2_block5_0_relu False
37 conv2_block

In [ ]:
# Compile
inception_model.compile(
    optimizer=tfk.optimizers.Nadam(2e-4),
    loss=tfk.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
for i in range(3):
  # Fine-tune the model
  inception_history = inception_model.fit(
      x = X_train,
      y = y_train,
      batch_size = 128,
      epochs = 30,
      validation_data = (X_val, y_val),
      callbacks=[
          tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True),
          tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
      ],
      shuffle = True
  ).history

  # Calculate and print the final validation accuracy
  final_val_accuracy = round(max(inception_history['val_accuracy'])* 100, 2)
  print(f'Final validation accuracy: {final_val_accuracy}%')

  # Save the trained model to a file with the accuracy included in the filename
  model_filename = 'inception_model_ft2.keras'
  inception_model.save(model_filename)

# Delete the model to free up resources
del inception_model

Epoch 1/30
588/588 ━━━━━━━━━━━━━━━━━━━━ 67s 71ms/step - accuracy: 0.8139 - loss: 0.5912 - val_accuracy: 0.9501 - val_loss: 0.1370 - learning_rate: 2.0000e-04
Epoch 2/30
588/588 ━━━━━━━━━━━━━━━━━━━━ 34s 57ms/step - accuracy: 0.8476 - loss: 0.4986 - val_accuracy: 0.9590 - val_loss: 0.1154 - learning_rate: 2.0000e-04
Epoch 3/30
588/588 ━━━━━━━━━━━━━━━━━━━━ 34s 57ms/step - accuracy: 0.8536 - loss: 0.4801 - val_accuracy: 0.9685 - val_loss: 0.0968 - learning_rate: 2.0000e-04
Epoch 4/30
588/588 ━━━━━━━━━━━━━━━━━━━━ 34s 57ms/step - accuracy: 0.8613 - loss: 0.4537 - val_accuracy: 0.9707 - val_loss: 0.0846 - learning_rate: 2.0000e-04
Epoch 5/30
588/588 ━━━━━━━━━━━━━━━━━━━━ 34s 57ms/step - accuracy: 0.8682 - loss: 0.4352 - val_accuracy: 0.9732 - val_loss: 0.0791 - learning_rate: 2.0000e-04
Epoch 6/30
588/588 ━━━━━━━━━━━━━━━━━━━━ 33s 57ms/step - accuracy: 0.8705 - loss: 0.4285 - val_accuracy: 0.9691 - val_loss: 0.0873 - learning_rate: 2.0000e-04
Epoch 7/30
588/588 ━━━━━━━━━━━━━━━━━━━━ 34s 57ms/ste

## 🥵 Third fine tuning


In [ ]:
# Re-load the model
inception_model = tfk.models.load_model('inception_model_ft2.keras')

for layer in inception_model.layers:
    print(layer.name, type(layer).__name__)

# Set the model layers as trainable
inception_model.get_layer('densenet121').trainable = True

# Set all layers as non-trainable
for layer in inception_model.get_layer('densenet121').layers:
    layer.trainable = False

# Enable training only for Conv2D and DepthwiseConv2D layers
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

input_layer InputLayer
preprocessing Sequential
rescaling Rescaling
densenet121 Functional
dropout Dropout
dense Dense
2 conv1_conv Conv2D True
9 conv2_block1_1_conv Conv2D True
12 conv2_block1_2_conv Conv2D True
16 conv2_block2_1_conv Conv2D True
19 conv2_block2_2_conv Conv2D True
23 conv2_block3_1_conv Conv2D True
26 conv2_block3_2_conv Conv2D True
30 conv2_block4_1_conv Conv2D True
33 conv2_block4_2_conv Conv2D True
37 conv2_block5_1_conv Conv2D True
40 conv2_block5_2_conv Conv2D True
44 conv2_block6_1_conv Conv2D True
47 conv2_block6_2_conv Conv2D True
51 pool2_conv Conv2D True
55 conv3_block1_1_conv Conv2D True
58 conv3_block1_2_conv Conv2D True
62 conv3_block2_1_conv Conv2D True
65 conv3_block2_2_conv Conv2D True
69 conv3_block3_1_conv Conv2D True
72 conv3_block3_2_conv Conv2D True
76 conv3_block4_1_conv Conv2D True
79 conv3_block4_2_conv Conv2D True
83 conv3_block5_1_conv Conv2D True
86 conv3_block5_2_conv Conv2D True
90 conv3_block6_1_conv Conv2D True
93 conv3_block6_2_conv Con

In [ ]:
# Set the number of layers to freeze
N = 50

# Set the first N layers as non-trainable
for i, layer in enumerate(inception_model.get_layer('densenet121').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    print(i, layer.name, layer.trainable)

0 input_layer False
1 zero_padding2d False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 zero_padding2d_1 False
6 pool1 False
7 conv2_block1_0_bn False
8 conv2_block1_0_relu False
9 conv2_block1_1_conv False
10 conv2_block1_1_bn False
11 conv2_block1_1_relu False
12 conv2_block1_2_conv False
13 conv2_block1_concat False
14 conv2_block2_0_bn False
15 conv2_block2_0_relu False
16 conv2_block2_1_conv False
17 conv2_block2_1_bn False
18 conv2_block2_1_relu False
19 conv2_block2_2_conv False
20 conv2_block2_concat False
21 conv2_block3_0_bn False
22 conv2_block3_0_relu False
23 conv2_block3_1_conv False
24 conv2_block3_1_bn False
25 conv2_block3_1_relu False
26 conv2_block3_2_conv False
27 conv2_block3_concat False
28 conv2_block4_0_bn False
29 conv2_block4_0_relu False
30 conv2_block4_1_conv False
31 conv2_block4_1_bn False
32 conv2_block4_1_relu False
33 conv2_block4_2_conv False
34 conv2_block4_concat False
35 conv2_block5_0_bn False
36 conv2_block5_0_relu False
37 conv2_block

In [ ]:
# Compile
inception_model.compile(
    optimizer=tfk.optimizers.Nadam(1e-4),
    loss=tfk.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
for i in range(3):
  # Fine-tune the model
  inception_history = inception_model.fit(
      x = X_train,
      y = y_train,
      batch_size = 256,
      epochs = 20,
      validation_data = (X_val, y_val),
      callbacks=[
          tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True),
          tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-8)
      ],
      shuffle = True
  ).history

  # Calculate and print the final validation accuracy
  final_val_accuracy = round(max(inception_history['val_accuracy'])* 100, 2)
  print(f'Final validation accuracy: {final_val_accuracy}%')

  # Save the trained model to a file with the accuracy included in the filename
  model_filename = 'densenet_model_ft3.keras'
  inception_model.save(model_filename)

# Delete the model to free up resources
del inception_model

Epoch 1/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 74s 133ms/step - accuracy: 0.8883 - loss: 0.3774 - val_accuracy: 0.9766 - val_loss: 0.0713 - learning_rate: 1.0000e-04
Epoch 2/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 29s 99ms/step - accuracy: 0.8959 - loss: 0.3585 - val_accuracy: 0.9808 - val_loss: 0.0555 - learning_rate: 1.0000e-04
Epoch 3/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 29s 99ms/step - accuracy: 0.9036 - loss: 0.3361 - val_accuracy: 0.9719 - val_loss: 0.0795 - learning_rate: 1.0000e-04
Epoch 4/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 29s 98ms/step - accuracy: 0.9068 - loss: 0.3297 - val_accuracy: 0.9746 - val_loss: 0.0779 - learning_rate: 1.0000e-04
Epoch 5/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 29s 99ms/step - accuracy: 0.9132 - loss: 0.3073 - val_accuracy: 0.9802 - val_loss: 0.0590 - learning_rate: 5.0000e-05
Epoch 6/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 29s 99ms/step - accuracy: 0.9183 - loss: 0.2956 - val_accuracy: 0.9780 - val_loss: 0.0602 - learning_rate: 5.0000e-05
Epoch 7/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 29s 99ms/st

## 🦂: Fourth fine tuning

In [ ]:
# Re-load the model
inception_model = tfk.models.load_model('densenet_model_ft3.keras')

for layer in inception_model.layers:
    print(layer.name, type(layer).__name__)

# Set the model layers as trainable
inception_model.get_layer('densenet121').trainable = True

# Set all layers as non-trainable
for layer in inception_model.get_layer('densenet121').layers:
    layer.trainable = False

# Enable training only for Conv2D and DepthwiseConv2D layers
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

input_layer InputLayer
preprocessing Sequential
rescaling Rescaling
densenet121 Functional
dropout Dropout
dense Dense
2 conv1_conv Conv2D True
9 conv2_block1_1_conv Conv2D True
12 conv2_block1_2_conv Conv2D True
16 conv2_block2_1_conv Conv2D True
19 conv2_block2_2_conv Conv2D True
23 conv2_block3_1_conv Conv2D True
26 conv2_block3_2_conv Conv2D True
30 conv2_block4_1_conv Conv2D True
33 conv2_block4_2_conv Conv2D True
37 conv2_block5_1_conv Conv2D True
40 conv2_block5_2_conv Conv2D True
44 conv2_block6_1_conv Conv2D True
47 conv2_block6_2_conv Conv2D True
51 pool2_conv Conv2D True
55 conv3_block1_1_conv Conv2D True
58 conv3_block1_2_conv Conv2D True
62 conv3_block2_1_conv Conv2D True
65 conv3_block2_2_conv Conv2D True
69 conv3_block3_1_conv Conv2D True
72 conv3_block3_2_conv Conv2D True
76 conv3_block4_1_conv Conv2D True
79 conv3_block4_2_conv Conv2D True
83 conv3_block5_1_conv Conv2D True
86 conv3_block5_2_conv Conv2D True
90 conv3_block6_1_conv Conv2D True
93 conv3_block6_2_conv Con

In [ ]:
# Set the number of layers to freeze
N = 10

# Set the first N layers as non-trainable
for i, layer in enumerate(inception_model.get_layer('densenet121').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    print(i, layer.name, layer.trainable)

0 input_layer False
1 zero_padding2d False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 zero_padding2d_1 False
6 pool1 False
7 conv2_block1_0_bn False
8 conv2_block1_0_relu False
9 conv2_block1_1_conv False
10 conv2_block1_1_bn False
11 conv2_block1_1_relu False
12 conv2_block1_2_conv True
13 conv2_block1_concat False
14 conv2_block2_0_bn False
15 conv2_block2_0_relu False
16 conv2_block2_1_conv True
17 conv2_block2_1_bn False
18 conv2_block2_1_relu False
19 conv2_block2_2_conv True
20 conv2_block2_concat False
21 conv2_block3_0_bn False
22 conv2_block3_0_relu False
23 conv2_block3_1_conv True
24 conv2_block3_1_bn False
25 conv2_block3_1_relu False
26 conv2_block3_2_conv True
27 conv2_block3_concat False
28 conv2_block4_0_bn False
29 conv2_block4_0_relu False
30 conv2_block4_1_conv True
31 conv2_block4_1_bn False
32 conv2_block4_1_relu False
33 conv2_block4_2_conv True
34 conv2_block4_concat False
35 conv2_block5_0_bn False
36 conv2_block5_0_relu False
37 conv2_block5_1_con

In [ ]:
# Compile
inception_model.compile(
    optimizer=tfk.optimizers.Nadam(8e-5),
    loss=tfk.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
for i in range(3):
  # Fine-tune the model
  inception_history = inception_model.fit(
      x = X_train,
      y = y_train,
      batch_size = 256,
      epochs = 20,
      validation_data = (X_val, y_val),
      callbacks=[
          tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True),
          tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-8)
      ],
      shuffle = True
  ).history

  # Calculate and print the final validation accuracy
  final_val_accuracy = round(max(inception_history['val_accuracy'])* 100, 2)
  print(f'Final validation accuracy: {final_val_accuracy}%')

  # Save the trained model to a file with the accuracy included in the filename
  model_filename = 'densenet_model_ft4.keras'
  inception_model.save(model_filename)

# Delete the model to free up resources
del inception_model

Epoch 1/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 70s 130ms/step - accuracy: 0.9097 - loss: 0.3193 - val_accuracy: 0.9744 - val_loss: 0.0757 - learning_rate: 8.0000e-05
Epoch 2/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 34s 114ms/step - accuracy: 0.9110 - loss: 0.3146 - val_accuracy: 0.9785 - val_loss: 0.0617 - learning_rate: 8.0000e-05
Epoch 3/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 33s 114ms/step - accuracy: 0.9151 - loss: 0.3046 - val_accuracy: 0.9788 - val_loss: 0.0585 - learning_rate: 8.0000e-05
Epoch 4/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 34s 114ms/step - accuracy: 0.9168 - loss: 0.3008 - val_accuracy: 0.9833 - val_loss: 0.0481 - learning_rate: 8.0000e-05
Epoch 5/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 33s 114ms/step - accuracy: 0.9173 - loss: 0.2916 - val_accuracy: 0.9760 - val_loss: 0.0664 - learning_rate: 8.0000e-05
Epoch 6/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 33s 114ms/step - accuracy: 0.9197 - loss: 0.2890 - val_accuracy: 0.9777 - val_loss: 0.0667 - learning_rate: 8.0000e-05
Epoch 7/20
294/294 ━━━━━━━━━━━━━━━━━━━━ 33s 11

## 💒 Fifth fine tuning (with randAugment only)

In [ ]:
del X_train
del y_train
data = np.load('training_augmented_big_randaugment.npz')
X_train_2 = data['X_train']
y_train_2 = data['y_train']
print(X_train_2.shape, y_train_2.shape)

(33484, 96, 96, 3) (33484, 8)


In [ ]:
# Re-load the model
inception_model = tfk.models.load_model('densenet_model_ft4.keras')

for layer in inception_model.layers:
    print(layer.name, type(layer).__name__)

# Set the model layers as trainable
inception_model.get_layer('densenet121').trainable = True

# Set all layers as non-trainable
for layer in inception_model.get_layer('densenet121').layers:
    layer.trainable = False

# Enable training only for Conv2D and DepthwiseConv2D layers
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

input_layer InputLayer
preprocessing Sequential
rescaling Rescaling
densenet121 Functional
dropout Dropout
dense Dense
2 conv1_conv Conv2D True
9 conv2_block1_1_conv Conv2D True
12 conv2_block1_2_conv Conv2D True
16 conv2_block2_1_conv Conv2D True
19 conv2_block2_2_conv Conv2D True
23 conv2_block3_1_conv Conv2D True
26 conv2_block3_2_conv Conv2D True
30 conv2_block4_1_conv Conv2D True
33 conv2_block4_2_conv Conv2D True
37 conv2_block5_1_conv Conv2D True
40 conv2_block5_2_conv Conv2D True
44 conv2_block6_1_conv Conv2D True
47 conv2_block6_2_conv Conv2D True
51 pool2_conv Conv2D True
55 conv3_block1_1_conv Conv2D True
58 conv3_block1_2_conv Conv2D True
62 conv3_block2_1_conv Conv2D True
65 conv3_block2_2_conv Conv2D True
69 conv3_block3_1_conv Conv2D True
72 conv3_block3_2_conv Conv2D True
76 conv3_block4_1_conv Conv2D True
79 conv3_block4_2_conv Conv2D True
83 conv3_block5_1_conv Conv2D True
86 conv3_block5_2_conv Conv2D True
90 conv3_block6_1_conv Conv2D True
93 conv3_block6_2_conv Con

In [ ]:
# Set the number of layers to freeze
N = 20

# Set the first N layers as non-trainable
for i, layer in enumerate(inception_model.get_layer('densenet121').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(inception_model.get_layer('densenet121').layers):
    print(i, layer.name, layer.trainable)

0 input_layer False
1 zero_padding2d False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 zero_padding2d_1 False
6 pool1 False
7 conv2_block1_0_bn False
8 conv2_block1_0_relu False
9 conv2_block1_1_conv False
10 conv2_block1_1_bn False
11 conv2_block1_1_relu False
12 conv2_block1_2_conv False
13 conv2_block1_concat False
14 conv2_block2_0_bn False
15 conv2_block2_0_relu False
16 conv2_block2_1_conv False
17 conv2_block2_1_bn False
18 conv2_block2_1_relu False
19 conv2_block2_2_conv False
20 conv2_block2_concat False
21 conv2_block3_0_bn False
22 conv2_block3_0_relu False
23 conv2_block3_1_conv True
24 conv2_block3_1_bn False
25 conv2_block3_1_relu False
26 conv2_block3_2_conv True
27 conv2_block3_concat False
28 conv2_block4_0_bn False
29 conv2_block4_0_relu False
30 conv2_block4_1_conv True
31 conv2_block4_1_bn False
32 conv2_block4_1_relu False
33 conv2_block4_2_conv True
34 conv2_block4_concat False
35 conv2_block5_0_bn False
36 conv2_block5_0_relu False
37 conv2_block5_1_

In [ ]:
# Compile
inception_model.compile(
    optimizer=tfk.optimizers.Nadam(8e-5),
    loss=tfk.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
for i in range(3):
  # Fine-tune the model
  inception_history = inception_model.fit(
      x = X_train_2,
      y = y_train_2,
      batch_size = 256,
      epochs = 30,
      validation_data = (X_val, y_val),
      callbacks=[
          tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=8, restore_best_weights=True),
          tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-8)
      ],
      shuffle = True
  ).history

  # Calculate and print the final validation accuracy
  final_val_accuracy = round(max(inception_history['val_accuracy'])* 100, 2)
  print(f'Final validation accuracy: {final_val_accuracy}%')

  # Save the trained model to a file with the accuracy included in the filename
  model_filename = 'densenet_model_def.keras'
  inception_model.save(model_filename)

# Delete the model to free up resources
del inception_model

Epoch 1/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 121s 577ms/step - accuracy: 0.9377 - loss: 0.1830 - val_accuracy: 0.9808 - val_loss: 0.0547 - learning_rate: 8.0000e-05
Epoch 2/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 118s 502ms/step - accuracy: 0.9403 - loss: 0.1747 - val_accuracy: 0.9844 - val_loss: 0.0408 - learning_rate: 8.0000e-05
Epoch 3/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 65s 499ms/step - accuracy: 0.9369 - loss: 0.1767 - val_accuracy: 0.9830 - val_loss: 0.0414 - learning_rate: 8.0000e-05
Epoch 4/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 84s 515ms/step - accuracy: 0.9380 - loss: 0.1731 - val_accuracy: 0.9805 - val_loss: 0.0502 - learning_rate: 8.0000e-05
Epoch 5/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 82s 516ms/step - accuracy: 0.9460 - loss: 0.1533 - val_accuracy: 0.9774 - val_loss: 0.0559 - learning_rate: 4.0000e-05
Epoch 6/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 82s 516ms/step - accuracy: 0.9487 - loss: 0.1440 - val_accuracy: 0.9677 - val_loss: 0.0952 - learning_rate: 4.0000e-05
Epoch 7/30
131/131 ━━━━━━━━━━━━━━━━━━━━ 80s 

## ♟ Evaluate InceptionResNetV2 Accuracy

In [ ]:
# Load the fine-tuned model for prediction on external images
inception_model = tfk.models.load_model('densenet_model_def.keras')

In [ ]:
# Predict class probabilities and get predicted classes
test_predictions = inception_model.predict(X_test, verbose=0)
test_predictions = np.argmax(test_predictions, axis=-1)

# Extract ground truth classes
test_gt = np.argmax(y_test, axis=-1)

# Calculate and display test set accuracy
test_accuracy = accuracy_score(test_gt, test_predictions)
print(f'Accuracy score over the test set: {round(test_accuracy, 4)}')

Accuracy score over the test set: 0.9816


## 🎷: Test Time Augmentation

In [ ]:
# Load the fine-tuned model for prediction on external images
inception_model = tfk.models.load_model('densenet_model_def.keras')

In [ ]:
model = inception_model

model.save('weights.keras')
del model
del inception_model

In [ ]:
%%writefile model.py
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

import keras_cv as kcv

import random

augmentation = tf.keras.Sequential([
    tfkl.RandomFlip("horizontal_vertical"),
    tfkl.RandomTranslation(0.25, 0.25),
    tfkl.RandomZoom(0.2),
    tfkl.RandomRotation(0.4),
    tfkl.RandomContrast(0.2),
    tfkl.RandomBrightness(0.2)
], name='preprocessing')

class Model:
    def __init__(self):
        self.neural_network = tfk.models.load_model('weights.keras')

    def predict(self, X):
        preds = self.neural_network.predict(X)
        for i in range(30):
            preds = preds + self.neural_network.predict(augmentation(X))
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds

Overwriting model.py


In [ ]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
!zip {filename} model.py weights.keras

from google.colab import files
files.download(filename)

  adding: model.py (deflated 55%)
  adding: weights.keras (deflated 17%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>